# Bayesian optimization with Gaussian Process regression (GPR) model

In this exercise the we will use the Gaussian Process regression (GPR) from previous exercises and apply it in a Baysian optimization setting. 

Rather than obtaining an accurate model, the goal of Bayesian optimization is to optimize some unknown function that cannot be optimised directly. Instead we can observe the output for any input and try to find the optimal point by training a model and using it to select the next point. This is especially useful when the unknown function is expensive or time consuming to evaluate. 

## Dependencies

First we import the dependencies.

If you are in Colab, you need to install the [BoTorch](https://botorch.org/) package by uncommenting and running the line `!pip3 install botorch` below before proceeding.

In [ ]:
# install dependencies
# !pip3 install botorch


In [ ]:
import matplotlib.pyplot as plt
import torch
import gpytorch
import botorch

torch.set_default_tensor_type(torch.DoubleTensor)


## Data

Again we use the Schwefel function to generate data. 

We first visualize the function on a grid of input points and then we sample the initial training dataset with a small amount of additive observation noise. Importantly, we can see that this function has several local optima, and therefore it can be difficult to optimize!

In [ ]:
def schwefel(x):
    """The Schwefel function has many local optima."""
    return 418.9829 * x.shape[-1] - (x * torch.sin(torch.sqrt(torch.abs(x)))).sum(dim=-1)

def noisy_schwefel(x, noise_std=1.0):
    """The Schwefel function with observation noise."""
    return schwefel(x) + noise_std * torch.randn(x.shape[0])

def standardize(y):
    """Standardize a vector to have zero mean and unit standard deviation."""
    return (y - y.mean()) / y.std()

# Define a grid of points on which to evaluate the function
n_grid = 100
levels = 30
x_min = torch.tensor([0, 0])
x_max = torch.tensor([430, 430])

x0 = torch.linspace(0, 1, n_grid)
x1 = torch.linspace(0, 1, n_grid)
g0, g1 = torch.meshgrid(x0, x1, indexing='xy')
x_grid = torch.stack((g0.reshape(-1), g1.reshape(-1)), 1)

y_grid = schwefel(x_grid * (x_max - x_min) + x_min)

plt.figure(figsize=(5,4))
plt.title("Schwefel function")
plt.contourf(x0.numpy(), x1.numpy(), y_grid.reshape(n_grid, n_grid).numpy(), levels=levels)
plt.colorbar()
plt.show()


In [ ]:
# Sample a training set and plot it
n_train = 10

torch.manual_seed(9)
x_train = torch.rand(n_train, 2)
y_train = noisy_schwefel(x_train * (x_max - x_min) + x_min)

plt.figure(figsize=(5,4))
plt.title('Training data')
plt.scatter(x_train[:,0], x_train[:,1], c=y_train)
plt.colorbar()
plt.xlim(0, 1); plt.ylim(0, 1)
plt.show()


## Model

This time we will use a GPR model from the BoTorch package. It is very similar to the model we used in previous exercises, but we can define and train it with only a few lines of code. On the other hand, it is hard to see what is going on "under the hood".

In [ ]:
# Fit GP model to data
gp = botorch.models.SingleTaskGP(x_train, standardize(y_train).unsqueeze(-1))
mll = gpytorch.mlls.ExactMarginalLogLikelihood(gp.likelihood, gp)
botorch.fit.fit_gpytorch_mll(mll);


In addition to the trained model, we need to define an acquisition function to help select new data points. Many different acquisition functions exist, but they generally use a combination of the prediction and the predicted uncertainty to identify a point that has good potential to be closer to the optimum. 

You can read more about the different acquisition functions included in the BoTorch package [here](https://botorch.org/docs/acquisition#analytic-acquisition-functions). Especially the [analytical acquisition functions](https://botorch.org/docs/acquisition#analytic-acquisition-functions) can be useful in this exercise. Try to read about the [UpperConfidenceBound (UCB)](https://botorch.org/api/acquisition.html#botorch.acquisition.analytic.UpperConfidenceBound) acquisition function that we use in the code below and see how it works. 

Below we define an acquisition function, evaluate it on the grid data and plot the result. 

In [ ]:
# Construct acquisition function
acqf = botorch.acquisition.UpperConfidenceBound(gp, beta=9.0, maximize=True)
# acqf = botorch.acquisition.ExpectedImprovement(gp, best_f=standardize(y_train).max(), maximize=True)

# Evaluate acquisition function on grid
with torch.no_grad():
    acq = acqf(x_grid.reshape((x_grid.shape[0], 1, x_grid.shape[1])))

plt.figure(figsize=(5,4))
plt.title("Acquisition function")
plt.contourf(x0.numpy(), x1.numpy(), acq.reshape(n_grid, n_grid).numpy(), levels=levels)
plt.colorbar()
plt.show()


Now we define the optimisation loop where we iteratively:
* Train a GPR model.
* Evaluate the the acquisition function
* Select the most promising new data point.
* Label the new data point.
* Add the new data point to the dataset. 
* Repeat. 

In [ ]:
# optimisation loop

def run_optimisation_loop(x_data, y_data, x_pool, n_steps, maximize=True):
    for i in range(n_steps):
        print(f"Step: {i+1}/{n_steps}")
        # train GP model
        gp = botorch.models.SingleTaskGP(x_data, standardize(y_data).unsqueeze(-1))
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(gp.likelihood, gp)
        botorch.fit.fit_gpytorch_mll(mll);
        # predict and evaluate acquisition function
        acqf = botorch.acquisition.UpperConfidenceBound(gp, beta=9.0, maximize=maximize)
        # acqf = botorch.acquisition.ExpectedImprovement(gp, best_f=standardize(y_data).max(), maximize=maximize)
        with torch.no_grad():
            acq = acqf(x_pool.reshape((x_pool.shape[0], 1, x_pool.shape[1])))
        index = torch.argmax(acq).item()
        x_candidate = x_pool[index].unsqueeze(0)
        # observe y by evaluating the schwefel function
        y_candidate = noisy_schwefel(x_candidate * (x_max - x_min) + x_min)
        # append candidate to the data set
        x_data = torch.cat((x_data, x_candidate))
        y_data = torch.cat((y_data, y_candidate))
    return x_data, y_data


# RUn optimisation loop
optimization_steps = 50
maximize = True  # Set to False for minimization
x_data, y_data = run_optimisation_loop(x_train.clone(), y_train.clone(), x_grid, optimization_steps, maximize)


The results are shown below. The observed target value is plotted for each iteration along with the best value observed so far. 

In [ ]:
if maximize:
    y_best = [y_data[:i+1].max() for i in range(len(y_data))]
else:
    y_best = [y_data[:i+1].min() for i in range(len(y_data))]


plt.figure(figsize=(6, 3))
plt.plot(range(optimization_steps-len(y_data)+1, optimization_steps+1), y_data, label="y")
plt.plot(range(optimization_steps-len(y_data)+1, optimization_steps+1), y_best, color="red", label="best y")
plt.axvline(0, color="black", linestyle="--")
plt.xlabel("Iteration"); plt.ylabel("y")
plt.legend()
plt.show()


We can also visualize the final dataset in a scatter plot. Did we find the optimal value?

In [ ]:
if maximize:
    x_best = x_data[torch.argmax(y_data).item()]
else:
    x_best = x_data[torch.argmin(y_data).item()]


plt.figure(figsize=(5,4))
plt.title("Data")
plt.scatter(x_data[:,0], x_data[:,1], c=y_data)
plt.colorbar()
plt.scatter(x_best[0], x_best[1], c="red", marker="*")
plt.xlim(0, 1); plt.ylim(0, 1)
plt.show()


## Additional Exercises:

* Try changing the initial amount of training data.
* Try optimising a different part of the Schwefel function by changing `x_min` and `x_max`.
* Try to use a different acquisitions function.
* Try to minimize instead of maximize the function.
* Compare the performance of several different acquisition functions.
* Rewrite the code to use gradient-based optimisation of the acquisition function using `botorch.optim.optimize_acqf`.